In [1]:
%%writefile jetson_global_exec.py
import paramiko
import sys
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--command", type=str, default="ls", help="linux command to execute globally")
parser.add_argument("--min", type=str, default="20101")
parser.add_argument("--max", type=str, default="20136")
args = parser.parse_args()
JETSON_IP = "147.47.200.209"
MIN_PORT = args.min
MAX_PORT = args.max
USERNAME = "jetson"
PASSWORD = "jetson"
COMMAND = args.command

class GlobalExecute:
    def __init__(self):
        self.cli = paramiko.SSHClient()
        self.cli.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
        
        self.clients = {}
        self.ping()

    def ping(self):
        self.available = []
        for port in range(int(MIN_PORT), int(MAX_PORT)+1):
            if port % 10 > 6 or port % 10 == 0:
                continue

            try:
                self.available.append(port)

                cli = paramiko.SSHClient()
                cli.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
                cli.connect(JETSON_IP, port=port, username=USERNAME, password=PASSWORD)

                stdin, stdout, stderr = cli.exec_command("ls")
                lines = stdout.readlines()
            
                if lines:
                    self.clients[port] = cli
                    print(f"{port} is working: ls {lines}")

                
            except Exception as e:
                print(f"{port} {e}")
                continue
    
        temp = [i for i in range(int(MIN_PORT), int(MAX_PORT)+1)]
        result = []

        for port in temp:
            if port % 10 <= 6 and port % 10 > 0:
                result.append(port)

        print("NOT CONNECTED PORTS : ", list(set(result)-set(self.available)))

    def global_exec(self, command):
        # command : linux command to execute 
        print(f"Executes '{command}'")
        for port in self.available:
            try:
                cli = self.clients[port]
                stdin, stdout, stderr = cli.exec_command(f"{command}")
            
                if stdout:
                    stdout = stdout.readlines()
                if stderr:
                    stderr = stderr.readlines()
            
                print(f"Port {port} OUTPUT {stdout}")
                
                if stderr:
                    print(f"Port {port} ERROR {stderr}")
            

            except Exception as e:
                print(f"Port {port} error {e}")
                continue


    
if __name__ == "__main__":
    cl = GlobalExecute()
    cl.global_exec(COMMAND)
    
    res = "y"
    while res == "y":
        res = input("More commands? (y/n)\n")
        if res == "y":
            new_command = input("new command:\n")
            new_command.strip("\n")
            cl.global_exec(str(new_command))
        
        else:
            for k, v in cl.clients.items():
                v.close()
            sys.exit()

Overwriting jetson_global_exec.py


In [3]:
!python jetson_global_exec.py --command "ls"

^C


In [18]:
import paramiko
from scp import SCPClient

file_path = "./ambient_fl.zip"
def createSSHClient(server, port, user, password):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(server, port, user, password)
    return client

MIN_PORT = 20136
MAX_PORT = 20101
for port in range(int(MIN_PORT), int(MAX_PORT)+1):
    if port % 10 > 6 or port % 10 == 0:
        continue

    try:
        ssh = createSSHClient("147.47.200.209", port, "jetson", "jetson")
        scp_cl = SCPClient(ssh.get_transport())
        scp_cl.put(file_path, recursive=True)


    except Exception as e:
        print(f"{port} {e}")
        continue
        

In [2]:
ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 8E3D-25A1

 C:\Users\david\Desktop\앰비언트\Practice # - FL\ambient_fl 디렉터리

2021-09-12  오후 05:10    <DIR>          .
2021-09-12  오후 05:10    <DIR>          ..
2021-09-12  오후 04:57    <DIR>          .ipynb_checkpoints
2021-09-12  오후 04:57    <DIR>          FL_Client
2021-09-12  오후 04:57    <DIR>          FL_Server
2021-09-12  오후 05:10             6,170 Guide.ipynb
2021-09-12  오후 04:58             3,054 jetson_global_exec.py
2021-09-12  오후 04:57                 0 README.md
2021-09-12  오후 04:57               375 requirements.txt
               4개 파일               9,599 바이트
               5개 디렉터리  424,961,101,824 바이트 남음
